In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import datetime
%matplotlib inline

pd.set_option("display.max_columns",100)

In [2]:
start = datetime.datetime.now()

## Data processing

Initial datasets of bot-bot reverts are stored in datasets/revert_data as TSV files compressed in bzip2 format.

In [3]:
!ls ../datasets/revert_data/

dewiki_20161001_reverted_bot2bot.tsv.bz2
enwiki_20161201_reverted_bot2bot.tsv.bz2
eswiki_20161001_reverted_bot2bot.tsv.bz2
frwiki_20161001_reverted_bot2bot.tsv.bz2
jawiki_20161001_reverted_bot2bot.tsv.bz2
ptwiki_20161001_reverted_bot2bot.tsv.bz2
zhwiki_20161001_reverted_bot2bot.tsv.bz2


In [4]:
!bunzip2 -k ../datasets/revert_data/*.bz2

In [5]:
glob.glob("../datasets/revert_data/??wiki_20161?01_*bot2bot.tsv")

['../datasets/revert_data/enwiki_20161201_reverted_bot2bot.tsv',
 '../datasets/revert_data/zhwiki_20161001_reverted_bot2bot.tsv',
 '../datasets/revert_data/frwiki_20161001_reverted_bot2bot.tsv',
 '../datasets/revert_data/jawiki_20161001_reverted_bot2bot.tsv',
 '../datasets/revert_data/eswiki_20161001_reverted_bot2bot.tsv',
 '../datasets/revert_data/ptwiki_20161001_reverted_bot2bot.tsv',
 '../datasets/revert_data/dewiki_20161001_reverted_bot2bot.tsv']

In [9]:
df_dict = {}
for filename in glob.glob("../datasets/revert_data/*bot2bot.tsv"):
    lang_code = filename[24:26]
    df_dict[lang_code] = pd.read_csv(filename, sep="\t")
    df_dict[lang_code] = df_dict[lang_code].drop_duplicates()

In [10]:
for lang, lang_df in df_dict.items():
    print(lang, len(lang_df))

ja 44990
es 88684
de 68922
en 500553
fr 96533
pt 70869
zh 51423


In [11]:
df_dict['en'][0:2].transpose()

,0,1
rev_id,273691771,136526894
rev_timestamp,20090227173507,20070607044209
rev_user,6505923,4534303
rev_user_text,Kbdankbot,PbBot
rev_page,5040439,3046554
rev_sha1,qj45ne2z4yfexmpaz5wfnbm2yrmqt4j,3xtnw7u4w9h6cg1smw97mqnr1en6a55
rev_minor_edit,False,False
rev_deleted,False,False
rev_parent_id,2.59117e+08,1.20932e+08
archived,False,False


### Combining into one tidy dataframe

In [12]:
df_all = df_dict['en'].copy()
df_all = df_all.drop(df_all.index, axis=0)

for lang, lang_df in df_dict.items():
    lang_df['language'] = lang
    df_all = pd.concat([df_all, lang_df])

In [13]:
df_all['language'].value_counts()

en    500553
fr     96533
es     88684
pt     70869
de     68922
zh     51423
ja     44990
Name: language, dtype: int64

### Namespace type

In [14]:
def namespace_type(item):
    if int(item) == 0:
        return 'article'
    elif int(item) == 14:
        return 'category'
    elif int(item) % 2 == 1:
        return 'other talk'
    else:
        return 'other page'

In [15]:
df_all['namespace_type'] = df_all['page_namespace'].apply(namespace_type)

In [16]:
df_all['namespace_type'].value_counts()

article       561197
category      182601
other page    113498
other talk     64678
Name: namespace_type, dtype: int64

### Datetime parsing

In [17]:
def get_year(timestamp):
    return timestamp.year

In [18]:
df_all['reverting_timestamp_dt'] = pd.to_datetime(df_all['reverting_timestamp'], format="%Y%m%d%H%M%S")

df_all['reverted_timestamp_dt'] = pd.to_datetime(df_all['rev_timestamp'], format="%Y%m%d%H%M%S")

df_all = df_all.set_index('reverting_timestamp_dt')

df_all['reverting_timestamp_dt'] = pd.to_datetime(df_all['reverting_timestamp'], format="%Y%m%d%H%M%S")

df_all['time_to_revert'] = df_all['reverting_timestamp_dt']-df_all['reverted_timestamp_dt']

df_all['time_to_revert_hrs'] = df_all['time_to_revert'].astype('timedelta64[s]')/(60*60)

df_all['time_to_revert_days'] = df_all['time_to_revert'].astype('timedelta64[s]')/(60*60*24)

df_all['reverting_year'] = df_all['reverting_timestamp_dt'].apply(get_year)

df_all['time_to_revert_days_log10'] = df_all['time_to_revert_days'].apply(np.log10)

df_all['time_to_revert_hrs_log10'] = df_all['time_to_revert_hrs'].apply(np.log10)

### Filter all datasets to the same time bounds: 2001 to 2016

In [19]:
df_all = df_all.ix["2001-01-01":"2016-12-31"]

#### Counts per year: en

In [20]:
df_all[df_all['language']=='en'].reverting_year.value_counts().sort_index()

2004         2
2005       131
2006      3119
2007     17042
2008     33114
2009     36423
2010     30890
2011     63407
2012     48042
2013    201562
2014     20594
2015     24597
2016     21630
Name: reverting_year, dtype: int64

#### Counts per year: all

In [21]:
df_all.reverting_year.value_counts().sort_index()

2004       302
2005      1598
2006      6357
2007     29425
2008     54989
2009     81548
2010     68500
2011    147002
2012    103119
2013    342624
2014     26733
2015     35413
2016     24364
Name: reverting_year, dtype: int64

## Other processing and metadata

### Comment parsing: removing text in brackets/parens

Function for removing text within square brackets or parentheses, which is useful for aggregating comment messages.

In [22]:
# by http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python

def remove_brackets(test_str):
    """
    Takes a string and returns that string with text in brackets and parentheses removed
    """
    
    test_str = str(test_str)
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
            
    return " ".join(ret.split())

In [23]:
df_all['reverting_comment_nobracket'] = df_all['reverting_comment'].apply(remove_brackets)

### botpair and botpair_sorted

In [24]:
def concat_botpair(row):
    return str(row['reverting_user_text']) + " rv " + str(row['rev_user_text'])

def sorted_botpair(row):
    """
    Returns a sorted list. list.sort() is locale dependent, but it doesn't matter
    because all we need is some consistent (if arbitrary) way of uniquely sorting.
    """
    
    return str(sorted([row['reverting_user_text'], row['rev_user_text']]))

In [25]:
df_all['botpair'] = df_all.apply(concat_botpair, axis=1)

In [26]:
df_all['botpair_sorted'] = df_all.apply(sorted_botpair, axis=1)

### Namespace type

In [27]:
def namespace_type(item):
    if int(item) == 0:
        return 'article'
    elif int(item) == 14:
        return 'category'
    elif int(item) % 2 == 1:
        return 'other talk'
    else:
        return 'other page'

In [28]:
df_all['namespace_type'] = df_all['page_namespace'].apply(namespace_type)

In [29]:
df_all['namespace_type'].value_counts()

article       561197
category      182601
other page    113498
other talk     64678
Name: namespace_type, dtype: int64

### Reverts per page per botpair

This analysis is also replicated in R in reverts_per_page_R.ipynb

In [30]:
gb_lpb = df_all.groupby(["language", "rev_page", "botpair"])
gb_lpb_s = df_all.groupby(["language", "rev_page", "botpair_sorted"])

In [31]:
df_lpb = pd.DataFrame(gb_lpb['rev_id'].count()).reset_index().rename(columns={"rev_id":"reverts_per_page_botpair"})
df_lpb[0:5]

,language,rev_page,botpair,reverts_per_page_botpair
0,de,61,RedBot rv EmausBot,1
1,de,81,DumZiBoT rv CarsracBot,1
2,de,81,MerlIwBot rv ZéroBot,1
3,de,82,Alecs.bot rv SieBot,1
4,de,101,Xqbot rv Dinamik-bot,1


In [32]:
df_lpb_s = pd.DataFrame(gb_lpb_s['rev_id'].count()).reset_index().rename(columns={"rev_id":"reverts_per_page_botpair_sorted"})
df_lpb_s[0:5]

,language,rev_page,botpair_sorted,reverts_per_page_botpair_sorted
0,de,61,"['EmausBot', 'RedBot']",1
1,de,81,"['CarsracBot', 'DumZiBoT']",1
2,de,81,"['MerlIwBot', 'ZéroBot']",1
3,de,82,"['Alecs.bot', 'SieBot']",1
4,de,101,"['Dinamik-bot', 'Xqbot']",1


In [33]:
df_all = pd.merge(df_all, df_lpb, how='left',
         left_on=["language", "rev_page", "botpair"],
         right_on=["language", "rev_page", "botpair"])

df_all = pd.merge(df_all, df_lpb_s, how='left',
         left_on=["language", "rev_page", "botpair_sorted"],
         right_on=["language", "rev_page", "botpair_sorted"])

### Check time to revert for negatives

In [34]:
len(df_all.query("time_to_revert_days < 0"))


3

In [35]:
len(df_all.query("time_to_revert_days > 0"))


921971

In [36]:
df_all.query("time_to_revert_days < 0").groupby("language")['rev_id'].count()


language
de    3
Name: rev_id, dtype: int64

In [37]:
df_all.query("time_to_revert_days > 0").groupby("language")['rev_id'].count()


language
de     68919
en    500553
es     88684
fr     96533
ja     44990
pt     70869
zh     51423
Name: rev_id, dtype: int64

### Cleaning

These are some weird cases from de.wikipedia.org which look to have a database error

In [38]:
len(df_all)

921974

In [39]:
remove = [113534501, 142947562, 142917006]
df_all = df_all[~df_all['rev_id'].isin(remove)]

In [40]:
len(df_all)

921971

### Final data format

In [41]:
len(df_all)

921971

In [42]:
df_all[0:2].transpose()

,0,1
archived,False,False
language,ja,ja
page_namespace,14,0
rev_deleted,False,False
rev_id,31654187,36768330
rev_minor_edit,True,True
rev_page,649447,10
rev_parent_id,3.1402e+07,3.67548e+07
rev_revert_offset,1,1
rev_sha1,hi8o3p1yka5v6fb8hdux0771qjxfewp,nvmvm6tmzlze06abk9tvb4xiy1tp2ph


## Output to file

In [43]:
df_all.to_pickle("../datasets/processed_data/df_all_2016.pickle")

In [44]:
end = datetime.datetime.now()

time_to_run = end - start
minutes = int(time_to_run.seconds/60)
seconds = time_to_run.seconds % 60
print("Total runtime: ", minutes, "minutes, ", seconds, "seconds")

Total runtime:  7 minutes,  53 seconds
